# Drupal 8

Con esta guía de Jupyter vamos a lograr instalar una versión reciente de Drupal, en un servidor Ubuntu 16.04 recién instalado.

Primero debe instalar los componentes del servidor "LEMP", inciciando por la instalación de la base de datos. Seguidamente descargamos el código de Drupal.

Por último, creamos la base de datos a ser utilizada por Drupal, y luego corremos el instalador Web.


### MariaDB
MySQL y MariaDB tienen algunas pequeñas diferencias pero para nuestro objetivo son técnicamente equivalente, por lo cual vamos a preferir MariaDB sobre MySQL.

A la hora de instalar la base de datos de MariaDB, la herramienta de Apt de Debian va a insistir en solicitarnos una contraseña para el usuario "root" de la base de datos, el cual vamos a utilizar para tareas de administración.

Para evitar que en esta guía se nos solicite contraseña, vamos a usar el comando ``debconf-set-selections`` para definir la contraseña de antemano.

Nota: El usuario "root" de MariaDB y el de Linux son completamente aparte.

In [ ]:
echo "mariadb-server mysql-server/root_password password greencore" | sudo debconf-set-selections
echo "mariadb-server mysql-server/root_password_again password greencore" | sudo debconf-set-selections
sudo apt-get -y install mariadb-server

### Nginx y Php-fpm

Con Nginx no podemos usar el paquete de Php que es módulo para Apache, pero podemos utilizar ya sea Php-fpm o HHVM. En este caso usaremos Php-fpm pero se recomienda realizar sus propias pruebas de desempeño y estabilidad en la aplicación para decidir cual se comporta mejor para su escenario.

Nota: No vamos a reiniciar el servicio de Nginx en este momento, sino hasta luego de editar archivos de configuración.

In [ ]:
sudo apt-get -y install nginx
sudo apt-get -y install php7.0-fpm php7.0-cli php7.0-gd php7.0-mysql php7.0-xml

### Cambios en Php

**Desde una terminal**, y utilizando su editor preferido, debe cambiar dos archivos de la siguiente forma:

**Archivo:** ``/etc/php/7.0/fpm/php.ini``

**Cambio:** Debemos activar compatibilidad entre Php y FastCGI, y la forma en como intercambian con URLs.

```nginx
cgi.fix_pathinfo=0
```

**Archivo:** ``/etc/nginx/sites-available/default``

**Cambio:** Vamos a activar soporte para Php, incluyendo un archivo de configuración. Vamos a eliminar el resto de configuración, y agregar estos parámetros:

```
server {
    server_name ejemplo.com;

    access_log /var/log/nginx/ejemplo.com-access.log;
    error_log /var/log/nginx/ejemplo.com-error.log;
    root /var/www/html/;

    location / {
        index index.php;
        try_files $uri $uri/ /index.php$is_args$args;
    }

location ~ \.php$ {
                include snippets/fastcgi-php.conf;
                fastcgi_pass unix:/run/php/php7.0-fpm.sock;
        fastcgi_param SCRIPT_FILENAME $document_root$fastcgi_script_name;
    }
}

```

Reiniciamos Nginx y mostramos el estado del servicio.

## Drupal

Descargamos la última versión de Drupal.

In [ ]:
sudo systemctl restart nginx
sudo systemctl restart php7.0-fpm
sudo systemctl status nginx | cat
sudo systemctl status php7.0-fpm | cat
wget -qc https://ftp.drupal.org/files/projects/drupal-8.3.5.tar.gz
ls -lh

Descomprimimos el código de Drupal dentro de /var/www/html que es el directorio default de Nginx.

En un sistema en producción se recomienda crear un directorio para cada virtualhost, o un único virtualhost para drupal y hacer uso de la función de "Multi tenant" para trabajar varios clientes con una sola instancia de la aplicación.

Importante que el código queda con derechos de **www-data**, usuario que utiliza Debian para el servicio de web de Nginx. Esto porque Drupal debe poder manipular archivos dentro de la carpeta.

In [ ]:
sudo tar xzf ~/drupal-8.3.5.tar.gz -C /var/www/
sudo rm -r /var/www/html
sudo mv /var/www/drupal-8.3.5 /var/www/html
sudo chown -R www-data:www-data /var/www/html
ls /var/www/html

### Creación de base de datos

Antes de ejecutar el instalador Web de Drupal debemos crear la base de datos y el usuario que usará la aplicación de Drupal.

Para ello vamos a utilizar el comando de mysql(1), definiendo la contraseña "greencore" en la misma línea de comando.

In [ ]:
sudo mysql -u root -pgreencore << EOF
CREATE DATABASE drupal;
CREATE USER 'drupal' IDENTIFIED BY 'greencore';
GRANT ALL ON drupal.* TO 'drupal';
EOF

### Instalador web

No es necesario manipular ningún archivo Php para definir la configuración de nuestro sitio. Visitamos el sitio web, donde nos recibe el instalador Web de Drupal. Este funciona en estilo "wizard" donde simplemente vamos contestando con cuidado las consultas que nos hace.

http://10.42.25.3X/core/install.php

En "Set up database"
- Base de datos "drupal"
- Usario "drupal"
- Contraseña "greencore"

En "Configuration site" puede usar valores que usaría en un sitio en producción.
Recuerde utilizar para este laboratorio el usuario "admin", con contraseña "greencore" para facilitar la revisión de errores o dudas.